In [1]:
# !pip install tqdm

In [2]:
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm
tqdm.pandas()

import os
from pprint import pprint
import re
from collections import defaultdict

In [3]:
DATA_ADDRESS = "./data"
os.listdir(DATA_ADDRESS)

['.gitkeep',
 'correlation_df_mean_no_clean.csv',
 'correlation_df_std_no_clean.csv',
 'df_5s.pkl',
 'df_5s_source.pkl',
 'df_merged.pkl',
 'df_merged_pivot.xlsx',
 'df_processed_descriptive_statistics.pickle',
 'df_processed_descriptive_statistics.pkl',
 'df_processed_descriptive_statistics_mel40_mfcc20.pkl',
 'df_processed_simple.pkl',
 'df_processed_stat_no_trim_cleaned_fft2048_mel128_mfcc20.pkl',
 'df_processed_stat_no_trim_cleaned_fft2048_mel40_mfcc17.pkl',
 'df_processed_stat_no_trim_cleaned_fft512_mel64_mfcc17.pkl',
 'df_processed_stat_no_trim_no_clean_fft2048_mel128_mfcc20.pkl',
 'df_processed_stat_no_trim_no_clean_fft512_mel128_mfcc17.pkl',
 'preprocessed',
 'spearman_correlation_df_mean_cleaned.csv',
 'spearman_correlation_df_mean_no_clean.csv',
 'spearman_correlation_df_std_cleaned.csv',
 'spearman_correlation_df_std_no_clean.csv',
 'VOiCES_devkit']

In [4]:
TRAIN_DIR = os.path.join(DATA_ADDRESS,'preprocessed','train')
os.listdir(TRAIN_DIR)

['mel_len15_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12500-12799.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2000-2499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2500-2999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3000-3499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3500-3999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4000-4499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4500-4999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_500-999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5000-5499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5500-5999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_6000-6499.pkl',
 'mel_len15_fft2048_mels1

In [5]:
TEST_DIR = os.path.join(DATA_ADDRESS,'preprocessed','test')
os.listdir(TEST_DIR)

['mel_len15_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2000-2499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2500-2999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3000-3499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3500-3999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4000-4499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4500-4999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_500-999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5000-5499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5500-5999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_6000-6399.pkl',
 'mel_len5_fft2048_mels128_mfcc17_0-41.pkl',
 'mel_len5_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len5_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len5_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mel_len5_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mel_len5_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mel_len5_fft2048_mels128_mfcc17_11500-11999.

## Load data

In [6]:
def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj


def load_dataset(dataset_dir,train_files):
    ls_X = []
    for k,v in tqdm(train_files.items()):  
        train_file = v['file_name']
        file_dir = os.path.join(dataset_dir, train_file)
        ls_X.append(load_pickle(file_dir))
    return np.concatenate(ls_X, axis=0)

def load_to_dataframe(X,y):
    X_df = pd.DataFrame(X)

    # Convert the 1D array y to a pandas Series
    y_series = pd.Series(y, name='target')

    # Concatenate the features and target into a single DataFrame
    data = pd.concat([X_df, y_series], axis=1)

    return data

### loading y

In [7]:
b = pd.read_pickle(os.path.join(DATA_ADDRESS,'df_5s.pkl'))

In [8]:
df_raw = pd.read_pickle(os.path.join(DATA_ADDRESS,'df_5s_source.pkl'))
df_raw

,origin_folder,speaker,category,source
0,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
1,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
2,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
3,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
4,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
...,...,...,...,...
2322,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...
2323,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...
2324,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...
2325,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...


In [9]:
df_raw[df_raw['category'] == 'train']

,origin_folder,speaker,category,source
0,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
1,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
2,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
3,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
4,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
...,...,...,...,...
2314,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...
2315,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...
2316,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...
2317,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...


In [10]:
y_train = np.array(df_raw[df_raw['category']=='train']['speaker']).astype('float32')
print(y_train.shape)
y_train

(1552,)


array([4967., 4967., 4967., ..., 1066., 1066., 1066.], dtype=float32)

In [11]:
y_test = np.array(df_raw[df_raw['category']=='test']['speaker']).astype('float32')
print(y_test.shape)
y_test

(775,)


array([2137., 2137., 2137., 2137., 2137., 2137., 2137., 2137., 2628.,
       2628., 2628., 2628., 2628., 2628., 2628., 2628., 1603., 1603.,
       1603., 1603., 1603., 1603., 1603., 1603., 1226., 1226., 1226.,
       1226., 1226., 1226., 1226., 5139., 5139., 5139., 5139., 5139.,
       5139., 5139., 5139., 7437., 7437., 7437., 7437., 7437., 7437.,
       7437.,  176.,  176.,  176.,  176.,  176.,  176.,  176.,  176.,
        118.,  118.,  118.,  118.,  118.,  118.,  118.,  118., 3214.,
       3214., 3214., 3214., 3214., 3214., 3214., 3214., 3816., 3816.,
       3816., 3816., 3816., 3816., 3816., 3816., 1263., 1263., 1263.,
       1263., 1263., 1263., 1263., 1263., 5731., 5731., 5731., 5731.,
       5731., 5731., 5731., 5731.,  192.,  192.,  192.,  192.,  192.,
        192.,  192., 7594., 7594., 7594., 7594., 7594., 7594., 7594.,
       7594.,  175.,  175.,  175.,  175.,  175.,  175.,  175.,  175.,
       3389., 3389., 3389., 3389., 3389., 3389., 3389., 5322., 5322.,
       5322., 5322.,

### X_train

In [12]:

file_prefix = 'mfcc_len5_fft2048_mels128_mfcc17_'

# List all files in the directory
pprint(os.listdir(TRAIN_DIR))




['mel_len15_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12500-12799.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2000-2499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2500-2999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3000-3499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3500-3999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4000-4499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4500-4999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_500-999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5000-5499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5500-5999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_6000-6499.pkl',
 'mel_len15_fft2048_mels1

In [13]:
# Filter files that start with the specified prefix
files = os.listdir(TRAIN_DIR)
train_files_mfcc = [file for file in files if file.startswith(file_prefix)]
pprint(train_files_mfcc)

['mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_12500-12999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_13000-13499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_13500-13999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_14000-14499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_14500-14999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_15000-15499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_15500-15999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_16000-16499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_16500-16999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_17000-17499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_17500-17999.pkl',
 'm

In [14]:
# Filter files that start with the specified prefix
files = os.listdir(TEST_DIR)
test_files_mfcc = [file for file in files if file.startswith(file_prefix)]
pprint(test_files_mfcc)

['mfcc_len5_fft2048_mels128_mfcc17_0-41.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_12500-12999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_13000-13499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_13500-13999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_14000-14499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_14500-14999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_15000-15499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_15500-15999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_16000-16499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_16500-16999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_17000-17499.pkl',
 'mfcc_len

In [15]:
# Regular expression to match two numbers at the end of the file name
pattern = re.compile(r'_(\d+)-(\d+)\.pkl$')
n_interval = 500

# Extract the numbers
dir_files = defaultdict(dict)
for file_name in train_files_mfcc:
    match = pattern.search(file_name)
    n1 = int(match.group(1))
    n2 = int(match.group(2))
    file_num = n1/n_interval
    dir_files[file_num] = {
        'begin':n1,
        'end':n2,
        'file_name' : file_name,
        'metadata':df_raw[df_raw['category'] == 'train'][n1:n2+1]
    }
sorted_dir_files_train = {k: dir_files[k] for k in sorted(dir_files)}
pprint(sorted_dir_files_train)

{0.0: {'begin': 0,
       'end': 499,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
       'metadata':                origin_folder speaker category  \
0    source-16k/train/sp4967    4967    train   
1    source-16k/train/sp4967    4967    train   
2    source-16k/train/sp4967    4967    train   
3    source-16k/train/sp4967    4967    train   
4    source-16k/train/sp4967    4967    train   
..                       ...     ...      ...   
782  source-16k/train/sp4586    4586    train   
783  source-16k/train/sp4586    4586    train   
784  source-16k/train/sp4586    4586    train   
785  source-16k/train/sp4586    4586    train   
794  source-16k/train/sp2294    2294    train   

                                                source  
0    5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...  
1    5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...  
2    5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...  
3    5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...  
4  

In [16]:
# Regular expression to match two numbers at the end of the file name
pattern = re.compile(r'_(\d+)-(\d+)\.pkl$')
n_interval = 500

# Extract the numbers
dir_files = defaultdict(dict)
for file_name in test_files_mfcc:
    match = pattern.search(file_name)
    n1 = int(match.group(1))
    n2 = int(match.group(2))
    file_num = n1/n_interval
    dir_files[file_num] = {
        'begin':n1,
        'end':n2,
        'file_name' : file_name,
        'metadata':df_raw[df_raw['category'] == 'test'][n1:n2+1]
    }
sorted_dir_files_test = {k: dir_files[k] for k in sorted(dir_files)}
pprint(sorted_dir_files_test)

{0.0: {'begin': 0,
       'end': 499,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
       'metadata':                origin_folder speaker category  \
8     source-16k/test/sp2137    2137     test   
9     source-16k/test/sp2137    2137     test   
10    source-16k/test/sp2137    2137     test   
11    source-16k/test/sp2137    2137     test   
12    source-16k/test/sp2137    2137     test   
...                      ...     ...      ...   
1435  source-16k/test/sp0166    0166     test   
1436  source-16k/test/sp0166    0166     test   
1437  source-16k/test/sp0166    0166     test   
1438  source-16k/test/sp0166    0166     test   
1455  source-16k/test/sp8419    8419     test   

                                                 source  
8     5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...  
9     5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...  
10    5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...  
11    5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC... 

In [17]:
X_train = load_dataset(TRAIN_DIR,sorted_dir_files_train)
X_train.shape

 35%|███▌      | 35/100 [00:00<00:00, 113.32it/s]

100%|██████████| 100/100 [00:00<00:00, 106.52it/s]


(49664, 17, 216)

In [18]:
X_test= load_dataset(TEST_DIR,sorted_dir_files_test)
X_test.shape

100%|██████████| 50/50 [00:00<00:00, 103.64it/s]


(24800, 17, 216)

In [19]:
df_train = load_to_dataframe(X_train.reshape(X_train.shape[0], -1),y_train)
print(df_train.shape)
# Check the first few rows of the DataFrame
print(df_train.head())

(49664, 3673)
           0          1          2          3          4          5  \
0 -80.977600 -80.419434 -79.915077 -79.821449 -79.740036 -79.881989   
1 -75.017464 -71.060478 -65.008240 -60.050758 -56.970104 -56.826725   
2 -83.417419 -83.293800 -83.408394 -83.713020 -83.887047 -83.888329   
3 -71.465614 -66.708954 -63.589741 -61.603786 -59.245041 -58.487385   
4 -55.037834 -53.793888 -54.851269 -57.155281 -59.371613 -61.002224   

           6          7          8          9  ...      3663      3664  \
0 -80.015327 -80.039238 -80.408401 -80.696159  ...  0.789637  0.596668   
1 -58.076405 -61.482307 -66.518311 -72.484703  ... -0.306891 -0.246717   
2 -83.877029 -83.771271 -83.767853 -83.794540  ... -0.791701  0.350824   
3 -57.167061 -53.935825 -51.339558 -49.302170  ...  0.023778  0.023779   
4 -61.387573 -63.297401 -64.819847 -66.793404  ... -1.076139 -0.573394   

       3665      3666      3667      3668      3669      3670      3671  \
0  0.574223 -0.277681 -1.604426 -2.5812

In [20]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,3663,3664,3665,3666,3667,3668,3669,3670,3671,target
0,-80.977600,-80.419434,-79.915077,-79.821449,-79.740036,-79.881989,-80.015327,-80.039238,-80.408401,-80.696159,...,0.789637,0.596668,0.574223,-0.277681,-1.604426,-2.581222,-2.807574,-1.728784,-0.533316,4967.0
1,-75.017464,-71.060478,-65.008240,-60.050758,-56.970104,-56.826725,-58.076405,-61.482307,-66.518311,-72.484703,...,-0.306891,-0.246717,-0.213964,0.116363,0.144263,0.032544,0.068943,0.166217,0.220161,4967.0
2,-83.417419,-83.293800,-83.408394,-83.713020,-83.887047,-83.888329,-83.877029,-83.771271,-83.767853,-83.794540,...,-0.791701,0.350824,0.193292,0.363183,0.191189,0.263955,-0.255743,-0.354151,-0.447814,4967.0
3,-71.465614,-66.708954,-63.589741,-61.603786,-59.245041,-58.487385,-57.167061,-53.935825,-51.339558,-49.302170,...,0.023778,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,4967.0
4,-55.037834,-53.793888,-54.851269,-57.155281,-59.371613,-61.002224,-61.387573,-63.297401,-64.819847,-66.793404,...,-1.076139,-0.573394,-0.198749,-0.536650,-0.596209,-0.691602,-1.221938,-1.176664,-1.081235,4967.0


In [21]:
df_test = load_to_dataframe(X_test.reshape(X_test.shape[0], -1),y_test)
print(df_test.shape)
# Check the first few rows of the DataFrame
print(df_test.head())

(24800, 3673)
           0          1          2          3          4          5  \
0 -77.335411 -76.044968 -75.780907 -76.207321 -76.968941 -78.054825   
1 -73.302490 -74.613892 -75.964531 -74.992409 -73.675034 -73.362015   
2 -70.028969 -70.884056 -71.527603 -70.559921 -69.526054 -68.636116   
3 -69.520584 -70.708282 -73.370262 -74.992615 -76.616226 -77.995964   
4 -71.714066 -68.675499 -67.467400 -66.962158 -67.511063 -67.628944   

           6          7          8          9  ...      3663      3664  \
0 -78.889557 -79.136742 -78.707405 -77.364380  ...  1.666882  1.499124   
1 -74.261330 -75.696556 -76.656960 -77.399261  ...  0.682681  0.048326   
2 -67.744370 -66.134888 -65.494141 -67.076927  ... -0.003192  0.087003   
3 -78.455650 -79.094627 -80.521629 -80.936478  ...  0.023778  0.023779   
4 -67.616135 -67.188828 -67.699387 -67.895012  ...  1.495647  0.805673   

       3665      3666      3667      3668      3669      3670      3671  \
0  1.059202  0.374428 -0.468886 -1.1692

In [22]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,3663,3664,3665,3666,3667,3668,3669,3670,3671,target
0,-77.335411,-76.044968,-75.780907,-76.207321,-76.968941,-78.054825,-78.889557,-79.136742,-78.707405,-77.364380,...,1.666882,1.499124,1.059202,0.374428,-0.468886,-1.169286,-0.854867,-0.729526,-0.158081,2137.0
1,-73.302490,-74.613892,-75.964531,-74.992409,-73.675034,-73.362015,-74.261330,-75.696556,-76.656960,-77.399261,...,0.682681,0.048326,-0.012902,0.545132,1.274721,1.936445,2.379305,1.697993,1.388240,2137.0
2,-70.028969,-70.884056,-71.527603,-70.559921,-69.526054,-68.636116,-67.744370,-66.134888,-65.494141,-67.076927,...,-0.003192,0.087003,0.376900,-0.297952,-0.694625,-0.153152,-0.107342,-0.534249,-0.441920,2137.0
3,-69.520584,-70.708282,-73.370262,-74.992615,-76.616226,-77.995964,-78.455650,-79.094627,-80.521629,-80.936478,...,0.023778,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,2137.0
4,-71.714066,-68.675499,-67.467400,-66.962158,-67.511063,-67.628944,-67.616135,-67.188828,-67.699387,-67.895012,...,1.495647,0.805673,0.509418,0.759485,0.382243,0.110432,-0.015470,-0.084720,0.296677,2137.0
